## Fetch and Load Data

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.En.csv')
df_test = pd.read_csv('/content/task_B_En_test.csv')
df_train = df
df.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df=df.dropna(subset=['tweet'])
df_test=df_test.dropna(subset=['text'])

In [ ]:
df.size

34670

In [ ]:
len(df[df['sarcastic']==1].values)

867

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0, 2, 2])
ax.bar(['sarcastic', 'non-sarcastic'],
       [len(df[df['sarcastic']==1].values), len(df[df['sarcastic']==0].values)])
plt.show()

In [ ]:
sarcasm=df['sarcasm'].sum()
irony=df['irony'].sum()
satire=df['satire'].sum()
understatement=df['understatement'].sum()
overstatement=df['overstatement'].sum()
rhetorical_question=df['rhetorical_question'].sum()

label_names = ['sarcasm', 'irony','satire', 'understatement','overstatement', 'rhetorical_question']

fig = plt.figure()
ax = fig.add_axes([0,0,1.5,1.5])
students = [sarcasm,irony,satire,understatement,overstatement,rhetorical_question]
ax.bar(label_names,students)
plt.show()
print(sarcasm,irony,satire,understatement,overstatement,rhetorical_question)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

### **Tokenization**

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tokens = df['tweet'].apply(lambda x: ' '.join([val for val in word_tokenize(x.lower()) if val not in stop_words])).values
tokens_test = df_test['text'].apply(lambda x: ' '.join([val for val in word_tokenize(x.lower()) if val not in stop_words])).values

#### **TFIDF and Count Vectorizer**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
xtrain_bow = bow_vectorizer.fit_transform(tokens)
xtest_bow = bow_vectorizer.transform(tokens_test)

tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
xtrain_tfidf = tfidf_vectorizer.fit_transform(tokens)
xtest_tfidf= tfidf_vectorizer.transform(tokens_test)

#### **Word2Vec**

In [ ]:
tokens = [val.split() for val in tokens]
tokens_test = [val.split() for val in tokens_test]

In [ ]:
import gensim

w2v_model = gensim.models.Word2Vec(
            tokens,
            vector_size=300,
            window=5,
            min_count=0,
            sg=1,
            hs=0,
            negative=10,
            seed=42)

w2v_model.train(tokens, total_examples= len(tokens), epochs=30)

(1184743, 1376250)

In [ ]:
w2v_model.init_sims(replace=True)

<ipython-input-16-c7757d71a30b>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [ ]:
words = set(w2v_model.wv.index_to_key)
xtrain_w2v = np.array([np.mean(np.nan_to_num(np.array([w2v_model.wv[i] if i in words else np.zeros((300,)) for i in ls], dtype=object)), axis=0)
                         for ls in tokens], dtype=object)
xtest_w2v = np.array([np.mean(np.nan_to_num(np.array([w2v_model.wv[i] if i in words else np.zeros((300,)) for i in ls ], dtype=object)), axis=0)
                         for ls in tokens_test], dtype=object)

#### **GloVe**

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

--2023-04-03 21:48:15--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2023-04-03 21:48:15--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  5.02MB/s    in 4m 45s  

2023-04-03 21:53:00 (5.09 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408563/1520408563]

Archive:  glove.twitter.27B.zip
  inflating: glove.twitter.27

In [ ]:
EMBEDDING_FILE = './glove.twitter.27B.200d.txt'
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
x_train = df['tweet'].values
x_test = df_test['text'].values

In [ ]:
max_features = 35000
maxlen = 200

from keras.utils import pad_sequences

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### **Machine Learning Methods**

In [ ]:
y_train = df[['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']].values
y_test = df_test[['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']].values
y_train.shape, y_test.shape

((3467, 6), (1400, 6))

In [ ]:
y_train=y_train.dropna()

#### **ML with TFIDF**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from xgboost import XGBClassifier
import xgboost as xgb

rf = RandomForestClassifier(class_weight='balanced_subsample')
grid_rf = GridSearchCV(rf, {'n_estimators':[300,700], 'max_depth':[7,11]})

xgb_classifier = XGBClassifier()
grid_xgb = GridSearchCV(xgb_classifier, {'n_estimators':[500,700],
                                         'max_depth':[7,11]})

lr = LogisticRegressionCV(max_iter=1000)
svm = SVC()

lr_model = lr.fit(xtrain_tfidf, y_train)
rf_model = grid_rf.fit(xtrain_tfidf, y_train)
xgb_model = grid_xgb.fit(xtrain_tfidf, y_train)
svm_model = svm.fit(x_train_tfidf, y_train)

y_pred_lr = lr_model.predict(xtest_tfidf)
y_pred_rf = rf_model.predict(xtest_tfidf)
y_pred_xgb = xgb_model.predict(xtest_tfidf)
y_pred_svm = svm_model.predict(xtest_tfidf)

y_pred_lr1 = [1 if prob > 0.5 else 0 for prob in y_pred_lr]
y_pred_rf1 = np.where(y_pred_rf > 0.5, 1, 0)
y_pred_xgb1 = np.where(y_pred_xgb > 0.5, 1, 0)
y_pred_svm1 = np.where(y_pred_svm > 0.5, 1, 0)

print("F1-Score: LR with tfidf:", f1_score(y_test, y_pred_lr1 , average="macro"))
print("F1-Score: RF with tfidf:", f1_score(y_test, y_pred_rf1 , average="macro"))
print("F1-Score: XGB with tfidf:", f1_score(y_test, y_pred_xgb1 , average="macro"))
print("F1-Score: SVM with tfidf:", f1_score(y_test, y_pred_svm1 , average="macro"))

### **ML with BOW**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from xgboost import XGBClassifier
import xgboost as xgb

rf = RandomForestClassifier(class_weight='balanced_subsample')
grid_rf = GridSearchCV(rf, {'n_estimators':[300,700], 'max_depth':[7,11]})

xgb_classifier = XGBClassifier()
grid_xgb = GridSearchCV(xgb_classifier, {'n_estimators':[500,700],
                                         'max_depth':[7,11]})

lr = LogisticRegressionCV(max_iter=1000)
svm = SVC()

lr_model = lr.fit(xtrain_bow, y_train)
rf_model = grid_rf.fit(xtrain_bow, y_train)
xgb_model = grid_xgb.fit(xtrain_bow, y_train)
svm_model = svm.fit(x_train_bow, y_train)

y_pred_lr = lr_model.predict(xtest_bow)
y_pred_rf = rf_model.predict(xtest_bow)
y_pred_xgb = xgb_model.predict(xtest_bow)
y_pred_svm = svm_model.predict(xtest_svm)

y_pred_lr1 = [1 if prob > 0.5 else 0 for prob in y_pred_lr]
y_pred_rf1 = np.where(y_pred_rf > 0.5, 1, 0)
y_pred_xgb1 = np.where(y_pred_xgb > 0.5, 1, 0)
y_pred_svm1 = np.where(y_pred_svm > 0.5, 1, 0)

print("F1-Score: LR with BOW:", f1_score(y_test, y_pred_lr1 , average="binary"))
print("F1-Score: RF with BOW:", f1_score(y_test, y_pred_rf1 , average="macro"))
print("F1-Score: XGB with BOW:", f1_score(y_test, y_pred_xgb1 , average="macro"))
print("F1-Score: SVM with svm:", f1_score(y_test, y_pred_svm1 , average="macro"))

### LSTM Methods

In [ ]:
x_train = df['tweet'].values
x_test = df_test['text'].values

y_train = df[['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']].values
y_test = df_test[['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']].values
x_train.shape, y_train.shape

((3467,), (3467, 6))

#### **LSTM with GloVe**

In [ ]:
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.models import Model

vocab_len = len(word_index)
emb_dim = 200

embedding_layer = Embedding(vocab_len,
                            maxlen,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)

#Bi-LSTM
model = Sequential([
    embedding_layer,
  Bidirectional(LSTM(emb_dim, return_sequences=True)),
  Dropout(0.5),
  Bidirectional(LSTM(emb_dim,)),
  Dropout(0.5),
  Dense(6, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

'''
#LSTM
model = Sequential([
    embedding_layer,
    LSTM(emb_dim, return_sequences=True),
    Dropout(0.5),
    LSTM(emb_dim,),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())
'''

'''


#Option 2

lstm_model = Sequential()
lstm_model.add(Embedding(vocab_len, emb_dim, input_len = maxlen, trainable = False, weights=[embedding_matrix]))
lstm_model.add(LSTM(128, return_sequences=False))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation = 'sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model.summary())
'''

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 200, 200)          2277200   
                                                                 
 bidirectional_15 (Bidirecti  (None, 200, 400)         641600    
 onal)                                                           
                                                                 
 dropout_18 (Dropout)        (None, 200, 400)          0         
                                                                 
 bidirectional_16 (Bidirecti  (None, 400)              961600    
 onal)                                                           
                                                                 
 dropout_19 (Dropout)        (None, 400)               0         
                                                                 
 dense_15 (Dense)            (None, 6)                

"\n\n\n#Option 2\n\nlstm_model = Sequential()\nlstm_model.add(Embedding(vocab_len, emb_dim, input_len = maxlen, trainable = False, weights=[embedding_matrix]))\nlstm_model.add(LSTM(128, return_sequences=False))\nlstm_model.add(Dropout(0.5))\nlstm_model.add(Dense(1, activation = 'sigmoid'))\nlstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nprint(lstm_model.summary())\n"

In [ ]:
history = model.fit(np.expand_dims(x_train,axis=1), y_train, batch_size=64, epochs=10)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

print(f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred))

#### **Bidirectional GRUs with Attention**

In [ ]:
from tensorflow.keras.layers import Layer
from keras import backend as K


class AttentionLayer(keras.layers.Layer):

    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        """

        self.supports_masking = True
        self.init = keras.initializers.get('glorot_uniform')

        self.W_regularizer = keras.regularizers.get(W_regularizer)
        self.b_regularizer = keras.regularizers.get(b_regularizer)

        self.W_constraint = keras.constraints.get(W_constraint)
        self.b_constraint = keras.constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionLayer, self).__init__(**kwargs)


    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape = (input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape = (input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True


    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None


    def call(self, x, mask=None):
        # TF backend doesn't support it
        # eij = K.dot(x, self.W)
        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                              K.reshape(self.W, (features_dim, 1))),
                        (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a

        return K.sum(weighted_input, axis=1)


    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


    def get_config(self):
        config = {'step_dim': self.step_dim}
        base_config = super(AttentionLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
GRU_DIM = 128

inp = keras.layers.Input(shape=(maxlen,))
x = keras.layers.Embedding(vocab_len, emb_dim, trainable=True)(inp)
x = keras.layers.Bidirectional(keras.layers.CuDNNGRU(GRU_DIM, return_sequences=True))(x)
x = AttentionLayer(maxlen)(x)
x = keras.layers.Dense(GRU_DIM*2, activation='relu')(x)
x = keras.layers.Dropout(rate=0.2)(x)
x = keras.layers.Dense(GRU_DIM, activation='relu')(x)
x = keras.layers.Dropout(rate=0.2)(x)

outp = keras.layers.Dense(6, activation='sigmoid')(x)

model1 = keras.models.Model(inputs=inp, outputs=outp)

model1.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_8 (Embedding)     (None, 200, 200)          2277200   
                                                                 
 bidirectional_14 (Bidirecti  (None, 200, 256)         253440    
 onal)                                                           
                                                                 
 attention_layer_2 (Attentio  (None, 256)              456       
 nLayer)                                                         
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dropout_16 (Dropout)        (None, 256)               0   

In [ ]:
history = model1.fit(x_train, y_train, batch_size=64, epochs=10)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report
y_pred = model.predict(X_test)
y_pred1 = np.where(y_pred > 0.5, 1, 0)

print(f1_score(y_test, y_pred1 , average="macro"))
print(classification_report(y_test, y_pred1))